# Process Eikonal solver results to build a database

In [3]:
import glob
import meshio
import yaml
import numpy as np
import pyvista as pv

pv.set_jupyter_backend("static")
pv.set_plot_theme("document")
plotter = pv.Plotter(off_screen=True)

files = sorted(glob.glob("data/*/*study.vtu", recursive="True"))

tf = []
avg_ori = []
D_i = []
D_w = []

with open("models.yaml") as f:
    meta_data = yaml.safe_load(f)

for file in files:
    # Process target file with re-ordering
    target = meshio.read(file)
    indices = np.lexsort(np.fliplr(target.points).T)
    reverse_indices = np.argsort(indices)
    points = target.points[indices]
    cells = []
    for cell_block in target.cells:
        cell_block.data = reverse_indices[cell_block.data]
        cells.append(cell_block)
    point_data = {}
    for label, pdata in target.point_data.items():
        point_data[label] = pdata[indices]
    # Interpolate node data to points
    for label, cdata_list in target.cell_data.items():
        if cdata_list[0].ndim > 1:
            dim = cdata_list[0].shape[1]
        else:
            dim = 1
        pvals = np.zeros((len(points), dim))
        pcounts = np.zeros((len(points), dim))
        for cdata, cell_block in zip(cdata_list, cells):
            for value, nodes in zip(cdata, cell_block.data):
                pvals[nodes] += value
                pcounts[nodes] += 1
        point_data[label] = pvals / pcounts

    ori = np.array([point_data[key] for key in target.cell_data.keys() if key != "EID"])
    point_data["avg_ori"] = np.mean(ori, axis=0)

    # Process source file with re-ordering
    source = meshio.vtk.read(file.replace(".vtu", ".vtk"))
    _, indices = np.unique(source.points, return_index=True, axis=0)
    for label, pdata in source.point_data.items():
        point_data[label] = pdata[indices]

    # Export combined data on mesh
    mesh = meshio.Mesh(points=points, cells=cells, point_data=point_data)
    filename_merged = file.replace(".vtu", "_merged.vtu")
    meshio.write(filename_merged, mesh)


    # Get thickness from metadata
    thickness = meta_data[file.split("/")[1]]["thickness"]

    # Create picture of that file
    plotter.clear()
    pv_mesh = pv.read(filename_merged)
    pv_surf = pv_mesh.extract_surface()
    pv_surf = pv_surf.extrude((0, 0, thickness), capping=True)
    pv_edges = pv_surf.extract_feature_edges()
    plotter.add_mesh(pv_surf, scalars="Fuellzeit")
    plotter.add_mesh(pv_edges, color="black", line_width=5)
    plotter.remove_scalar_bar()
    filename_img = file.replace(".vtu", ".png")
    plotter.screenshot(filename_img, scale=2, transparent_background=True)

    # Append point-wise data
    tf.append(point_data["Fuellzeit"].flatten())
    avg_ori.append(point_data["avg_ori"])
    D_i.append(point_data["D_i"].flatten())
    D_w.append(point_data["D_w"].flatten())

# Combine data points
target = np.column_stack([np.concatenate(tf), np.concatenate(avg_ori)[:, 0:6]])
source = np.column_stack([np.concatenate(D_i), np.concatenate(D_w)])
mask = np.any(np.isnan(target), axis=1)
target = target[~mask]
source = source[~mask]

# Save data
np.savez_compressed("data.npz", target=target, source=source)
